# 3.4 softmax回归
回归可以预测多少的问题，但是同时，我们也对分类问题感兴趣：不是问”多少“，而是问”哪一个“。通常，机器学习用<b>分类</b>这个词来描述两个有微妙差别的问题：<br>
&emsp;(1)“硬性”类别，属于哪个类别；
&emsp;(2)“软性”类被，属于某个类别的概率。
这两者的界限往往很模糊，其中的一个原因是：即使我们只关心硬性类别，我们也依然使用软性类别的模型。
### 3.4.1 分类问题
从一个图像分类问题开始。假设每次输入是一个2像素*2像素的灰度像素。我们可以用一个标量表示每个像素值，每张图像对应4个特征$x_1$，$x_2$，$x_3$，$x_4$。此外，假设每张图像属于类别猫、狗、鸡中的一个。
接下来，我们要选择如何表示标签。我们有两个明显的选择。最直接的想法是选择y∈{1,2,3}，其中整数分别对应猫、狗、鸡。这是在计算机上存储此类信息的有效方法。如果类别间有了一定的自然顺序，例如我们试图预测{婴儿,儿童,青少年,中年人,老年人}，那么这个问题可以转变为回归问题，并且保留这种格式是有意义的。
但是一般的分类问题并不与类别之间的自然顺序有关。统计学家很早就发现了一种表示分类数据的简单方法：<b>热度编码(one-hot encoding)</b>。热度编码是一个向量，它的分量和类别一样多。类别分别对应的分量设置为1，其他所有分量设置为0.在我们的例子中，标签y将是一个三维向量，其中(1,0,0)代表猫，(0,1,0)代表狗，(0,0,1)代表鸡：
<center>y∈{(1,0,0),(0,1,0),(0,0,1)}</center>

### 3.4.2 网络架构
为了估计所有可能类别的条件概率，我们需要一个有多个输出的模型，每个类别对应一个输出。为了解决线性模型的分类问题，我们需要输出一样多的<b>仿射函数</b>。每个输出对应于它自己的仿射函数。在我们的例子中，由于我们有4个特征和3个可能的输出类别，我们需要12个标量来表示权重(带下标的w)，3个标量来表示偏置(带下标的b)。下面我们为每个输入计算3个为规范化的预测(logit)：$o_1$，$o_2$和$o_3$：
<center>$o_1$ = $x_1$$w_{11}$ + $x_2$$w_{12}$ + $x_3$$w_{13}$ + $x_4$$w_{14}$ + $b_1$</center>
<center>$o_2$ = $x_1$$w_{21}$ + $x_2$$w_{22}$ + $x_3$$w_{23}$ + $x_4$$w_{24}$ + $b_2$</center>
<center>$o_3$ = $x_1$$w_{31}$ + $x_2$$w_{32}$ + $x_3$$w_{33}$ + $x_4$$w_{34}$ + $b_3$</center>

我们可以用神经网络图来描述这个计算过程。与线性回归一样，softmax回归也是一个单层神经网络。由于计算每个输出$o_i$取决于所有输入$x_i$，因此softmax回归的输出层也是全连接层。
<center><img src="../image/softmax.jpg"></center>

为了更简洁地表达模型，我们仍然使用线性代数符号。通过向量形式表示为<b>o=Wx+b</b>，这是一种更适合数学和编写代码的形式。因此我们已经将所有权重放到一个3*4矩阵中。对于给定数据样本的特征x，我们的输出是由权重与输入特征进行矩阵-向量乘法加上偏置b得到的。

### 3.4.3 全连接层的参数开销
正如哦我们将在后续看到的，在深度学习中，全连接层无处不在。然而，顾名思义，全连接层是“完全”连续的，可能有很多可学习的参数。具体来说，对于任何具有d个输入和q个输出的全连接层，参数开销为O(dq)，这个数字在实践中可能高得令人望而却步。幸运的是，将d个输入转换为q个输出的成本可以减少到O(dq/n),其中超参数n可以由我们灵活指定，以在实际应用中在参数节省和有效性之间进行权衡。

### 3.4.4 softmax运算
现在我们将优化参数以最大化观测数据的概率。为了得到预测结果，我们将设置一个阈值，如选择具有最大概率的标签。
我们希望模型的输出$y*_j$可以视为属于类j的概率，然后选择具有最大输出值的类别$argmax_jy_j$作为我们的预测。例如，如果$y*_1$、$y*_2$和$y*_3$分别为0.1、0.8和0.1，那么我们预测类2，在我们的例子里代表“狗”。
然而我们不能将为规范化的预测o直接视作我们感兴趣的输出，应为将线性层的输出直接视为概率时存在一些问题：一方面。我们没有限制这些输出数值的总和为1，另一方面，根据输入的不同，输出可能为负值。这些都违背了概率论的基本公理。
要将输出视为概率，我们必须保证在任何数据上的输出都是非负的并且总和为1.此外，我们需要一个训练的目标函数，来激励模型精准地估计概率。例如，在分类器输出0.5的所有样本中，我们希望这些样本中刚好有一半实际上属于预测的类。这个属性叫做<b>校准(calibration)</b>。
